In [1]:
import pandas as pd

In [9]:
data = pd.read_csv("../00_data/merged_all.csv", sep=";", decimal=",")
data.head()

,Unnamed: 0,date,hour,day_ahead_price,intraday_price,price_diff,DE_solar_generation_forecast,DE_solar_generation_actual,DE_solar_generation_error,DE_wind_generation_forecast,DE_wind_generation_actual,DE_wind_generation_error
0,0,01.01.2016,'00 - 01,23.86,29.34,-5.48,0,0,0,7653,8648,995
1,1,01.01.2016,'01 - 02,22.39,24.28,-1.89,0,0,0,7421,8579,1158
2,2,01.01.2016,'02 - 03,20.59,25.26,-4.67,0,0,0,7266,8542,1276
3,3,01.01.2016,'03 - 04,16.81,20.24,-3.43,0,0,0,7180,8443,1263
4,4,01.01.2016,'04 - 05,17.41,21.94,-4.53,0,0,0,7016,8295,1279
